In [137]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [138]:
df1 = pd.read_csv('monday.csv', sep = ";")
df2 = pd.read_csv('tuesday.csv', sep = ";")
df3 = pd.read_csv('wednesday.csv', sep = ";")
df4 = pd.read_csv('thursday.csv', sep = ";")
df5 = pd.read_csv('friday.csv', sep = ";")

In [139]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [140]:
df_unique = pd.read_csv('df_unique.csv', index_col=None)
df_unique.drop('Unnamed: 0', axis=1, inplace=True)

In [141]:
# 5. Simulate a single customer
# 5.1 create transition probability matrix

In [142]:
# create table with transition for each timestamp 
# two columns (from-->to)
# concat/append this yourney for each customer
# count individual transitions from each location 
# (from fruit -> drinks (87 people in total)
# (from fruit -> spices (45 people in total) ...
# those are the probabilities for each location)

In [143]:
df_journey = df_unique.groupby(['customer_unique', 'location', 'date']).count().sort_values(by = ['customer_unique', 'date'])

In [144]:
entry_point = df_journey.reset_index(1).index.get_level_values(level=0).unique()

In [145]:
entry_list= []
for i in entry_point:
    entry_list.append(df_journey.reset_index(1).xs(i)['location'][0])

In [146]:
df_journey.reset_index(inplace=True)

In [147]:
# count entry location
import collections
entry_dict = collections.Counter(entry_list)

In [148]:
entry_p = []
for i in list(entry_dict.values()):
    entry_p.append(i/(sum(entry_dict.values())))

In [149]:
states_ = ['fruit', 'dairy', 'spices', 'drinks']

In [150]:
entry_dic = zip(states_,entry_p)
list(entry_dic)

[('fruit', 0.3774345198119543),
 ('dairy', 0.2875755540631296),
 ('spices', 0.18146406984553393),
 ('drinks', 0.15352585627938214)]

In [151]:
sum(entry_p)

1.0

In [152]:
df_trans = df_journey[['location']]

In [153]:
df_trans['_to']  = df_journey[['location']].shift(periods=-1, fill_value='checkout')

In [154]:
indexlist = df_trans[df_trans['location'] == 'checkout'].index
df_trans.drop(indexlist, inplace=True)
df_trans['transition'] = df_trans['location'] + "_" + df_trans['_to']

In [224]:
df_trans

,location,_to,transition,trans_fac
0,fruit,checkout,fruit_checkout,0
2,dairy,dairy,dairy_dairy,1
3,dairy,dairy,dairy_dairy,1
4,dairy,dairy,dairy_dairy,1
5,dairy,dairy,dairy_dairy,1
6,dairy,dairy,dairy_dairy,1
7,dairy,drinks,dairy_drinks,2
8,drinks,checkout,drinks_checkout,3
10,spices,spices,spices_spices,4
11,spices,spices,spices_spices,4


In [156]:
labels, trans = pd.factorize(df_trans['transition'])
df_trans['trans_fac'] = labels

In [157]:
len(labels)

46486

In [158]:
trans_dict = df_trans['transition'].value_counts(ascending=True)

In [159]:
values = [value for (key, value) in sorted(trans_dict.items())]
keys = [key for (key, value) in sorted(trans_dict.items())]
sorted_trans_dict = dict(zip(keys, values))
sorted_trans_dict

{'dairy_checkout': 1825,
 'dairy_dairy': 13098,
 'dairy_drinks': 1044,
 'dairy_fruit': 891,
 'dairy_spices': 916,
 'drinks_checkout': 2096,
 'drinks_dairy': 106,
 'drinks_drinks': 5822,
 'drinks_fruit': 856,
 'drinks_spices': 846,
 'fruit_checkout': 2555,
 'fruit_dairy': 1221,
 'fruit_drinks': 699,
 'fruit_fruit': 7587,
 'fruit_spices': 646,
 'spices_checkout': 941,
 'spices_dairy': 1213,
 'spices_drinks': 1024,
 'spices_fruit': 575,
 'spices_spices': 2525}

In [160]:
df_unique.head()

,date,location,weekday,customer_unique,year,month,day,hour,minute,customer_time_spend
0,2019-09-06 17:19:00,fruit,Friday,1000Friday,2019,9,6,17,19,0 days 00:01:00.000000000
1,2019-09-06 17:20:00,checkout,Friday,1000Friday,2019,9,6,17,20,0 days 00:01:00.000000000
2,2019-09-02 17:44:00,dairy,Monday,1000Monday,2019,9,2,17,44,0 days 00:07:00.000000000
3,2019-09-02 17:45:00,dairy,Monday,1000Monday,2019,9,2,17,45,0 days 00:07:00.000000000
4,2019-09-02 17:46:00,dairy,Monday,1000Monday,2019,9,2,17,46,0 days 00:07:00.000000000


In [161]:
trans_matrix = pd.DataFrame.from_dict({'entrance': [0,0,0,0,0,0],
                'fruit': [0,0,0,0,0,0], 
                'spices': [0,0,0,0,0,0],
                'dairy': [0,0,0,0,0,0],
                'drinks': [0,0,0,0,0,0],
                'checkout': [0,0,0,0,0,0]})
trans_matrix.set_index([pd.Index(['entrance', 'fruit', 'spices', 'dairy', 'drinks', 'checkout'])], inplace=True)

In [162]:
trans_matrix

,entrance,fruit,spices,dairy,drinks,checkout
entrance,0,0,0,0,0,0
fruit,0,0,0,0,0,0
spices,0,0,0,0,0,0
dairy,0,0,0,0,0,0
drinks,0,0,0,0,0,0
checkout,0,0,0,0,0,0


In [163]:
for f in sorted_trans_dict.items():
    origin = str(f[0].split('_')[0])
    dest = str(f[0].split('_')[1])
    count_ = int(f[1])
    trans_matrix.loc[origin][dest] = count_

In [164]:
row_sum = trans_matrix.sum(axis=1)

In [165]:
prob_matrix = trans_matrix.T/row_sum
prob_matrix.loc['checkout']['checkout']=1
#prob_matrix.loc['entrance']['entrance']=1
prob_matrix

,entrance,fruit,spices,dairy,drinks,checkout
entrance,NaN,0.000000,0.000000,0.000000,0.000000,NaN
fruit,NaN,0.597025,0.091590,0.050129,0.088012,NaN
spices,NaN,0.050834,0.402198,0.051536,0.086983,NaN
dairy,NaN,0.096081,0.193214,0.736919,0.010899,NaN
drinks,NaN,0.055005,0.163109,0.058737,0.598602,NaN
checkout,NaN,0.201054,0.149888,0.102678,0.215505,1.0


In [166]:
prob_matrix.fillna(0, inplace=True)

In [167]:
prob_dict = dict(prob_matrix)
# check:
prob_matrix.T.loc['spices'].sum()

1.0

In [168]:
prob_matrix

,entrance,fruit,spices,dairy,drinks,checkout
entrance,0.0,0.000000,0.000000,0.000000,0.000000,0.0
fruit,0.0,0.597025,0.091590,0.050129,0.088012,0.0
spices,0.0,0.050834,0.402198,0.051536,0.086983,0.0
dairy,0.0,0.096081,0.193214,0.736919,0.010899,0.0
drinks,0.0,0.055005,0.163109,0.058737,0.598602,0.0
checkout,0.0,0.201054,0.149888,0.102678,0.215505,1.0


In [169]:
# add 0 probability for going to checkout from entrance
entry_p.append(0)
entry_p.insert(0,0)

In [172]:
entry_p

[0,
 0.3774345198119543,
 0.2875755540631296,
 0.18146406984553393,
 0.15352585627938214,
 0]

In [173]:
# adding entrance probability
prob_matrix['entrance'] = entry_p

In [195]:
prob_matrix

,entrance,fruit,spices,dairy,drinks,checkout
entrance,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
fruit,0.377435,0.597025,0.091590,0.050129,0.088012,0.0
spices,0.287576,0.050834,0.402198,0.051536,0.086983,0.0
dairy,0.181464,0.096081,0.193214,0.736919,0.010899,0.0
drinks,0.153526,0.055005,0.163109,0.058737,0.598602,0.0
checkout,0.000000,0.201054,0.149888,0.102678,0.215505,1.0


In [175]:
# Average number of steps during journey

In [176]:
# counting every stage (including ideling customers)
(df_journey.shape[0]-1)/df_journey['customer_unique'].nunique()

7.24002686366689

In [177]:
df_journey['customer_unique'].nunique()

7445

In [178]:
len(set(df_journey['customer_unique'].values))

7445

In [179]:
# counting every different stage

In [180]:
df.shape[0]/df_journey['customer_unique'].nunique()

3.3414372061786435

In [181]:
# calculate probabilities at entrance
# by counting first stage of all unique customers
# cal. percentage value
# use this number in markov.next_state('XXX') function 

In [183]:
x = prob_dict.keys()
list(x)[5]

'checkout'

_______________________________

In [225]:
class Customer(object):
    def __init__(self, transition_prob):
        """
        Initialize the MarkovChain instance.
        Parameters
        ----------
        transition_prob: dict
            A dict object representing the transition
            probabilities in Markov Chain.
            Should be of the form:
                {'state1': {'state1': 0.1, 'state2': 0.4},
                 'state2': {...}}
        """
        self.transition_prob = transition_prob
        self.states = list(transition_prob.keys())
        self.first_state = 'entrance'

    def next_state(self, current_state):
        return np.random.choice(
            self.states,
            p=[self.transition_prob[current_state][next_state]
               for next_state in self.states])

    def generate_states(self, current_state='entrance', no=50):
        future_states = []
        for i in range(no):
            next_state = self.next_state(current_state)
            future_states.append(next_state)
            current_state = next_state
            if future_states[-1] == self.states[-1]:
                break
        return future_states

In [226]:
markow = Customer(prob_dict)

In [229]:
markow.generate_states()

['spices',
 'fruit',
 'fruit',
 'fruit',
 'spices',
 'spices',
 'drinks',
 'drinks',
 'checkout']

In [223]:
markov.transition_prob

{'entrance': entrance    0.000000
 fruit       0.377435
 spices      0.287576
 dairy       0.181464
 drinks      0.153526
 checkout    0.000000
 Name: entrance, dtype: float64, 'fruit': entrance    0.000000
 fruit       0.597025
 spices      0.050834
 dairy       0.096081
 drinks      0.055005
 checkout    0.201054
 Name: fruit, dtype: float64, 'spices': entrance    0.000000
 fruit       0.091590
 spices      0.402198
 dairy       0.193214
 drinks      0.163109
 checkout    0.149888
 Name: spices, dtype: float64, 'dairy': entrance    0.000000
 fruit       0.050129
 spices      0.051536
 dairy       0.736919
 drinks      0.058737
 checkout    0.102678
 Name: dairy, dtype: float64, 'drinks': entrance    0.000000
 fruit       0.088012
 spices      0.086983
 dairy       0.010899
 drinks      0.598602
 checkout    0.215505
 Name: drinks, dtype: float64, 'checkout': entrance    0.0
 fruit       0.0
 spices      0.0
 dairy       0.0
 drinks      0.0
 checkout    1.0
 Name: checkout, dtype: fl